In [25]:
import scipy.signal as sig
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.io as sio

def group_delay(ww, phase):
    groupDelay = -np.diff(phase)/np.diff(ww)    
    return(np.append(groupDelay, groupDelay[-1]))

fig_sz_x = 14
fig_sz_y = 13
fig_dpi = 80 # dpi

fig_font_size = 16

plt.rcParams.update({'font.size':fig_font_size})

fs = 1000 # Hz
nyq_frec = fs / 2

# filter design
ripple = 0.5 # dB
atenuacion = 40 # dB

ws1 = 1.0 #Hz
wp1 = 3.0 #Hz
wp2 = 15.0 #Hz
ws2 = 35.0 #Hz

frecs = np.array([0.0, ws1, wp1, wp2, ws2, nyq_frec]) / nyq_frec
gains = np.array([-atenuacion, -atenuacion, -ripple, -ripple, -atenuacion, -atenuacion])
gains = 10**(gains/20)

bp_sos_butter = sig.iirdesign(wp=np.array([wp1, wp2]) / nyq_frec, ws=np.array([ws1, ws2]) / nyq_frec, gpass=0.5, gstop=40., analog=False, ftype='butter', output='sos')

cant_coef = 5001

num_fir = sig.firwin2(cant_coef, frecs, gains, window='blackmanharris')

den = 1.0

plt.rcParams.update({'font.size':fig_font_size})

w_iir, h_iir = sig.sosfreqz(bp_sos_butter)
phase_iir = np.angle(h_iir)
gd_iir = group_delay(w_iir, phase_iir)

w_fir, h_fir = sig.freqz(num_fir, den)
phase_fir = np.angle(h_fir)
gd_fir = group_delay(w_fir, phase_fir)

plt.figure(figsize=(fig_sz_x, fig_sz_y), dpi= fig_dpi, facecolor='w', edgecolor='k')

plt.plot(w_iir/ np.pi * nyq_frec, 20 * np.log10(abs(h_iir)), label='IIR-Butter' )
plt.plot(w_fir/ np.pi * nyq_frec, 20 * np.log10(abs(h_fir)), label='FIR-Win')
plt.plot(frecs * nyq_frec, 20*np.log10(gains), 'rx', label='plantilla' )

plt.title('Filtros diseñado por métodos directos')
plt.xlabel('Frequencia [Hz]')
plt.ylabel('Modulo [dB]')
plt.axis([0, nyq_frec, -60, 5 ])

plt.grid()

axes_hdl = plt.gca()
axes_hdl.legend()

angles_iir = np.unwrap(phase_iir)
angles_fir = np.unwrap(phase_fir)

plt.figure(figsize=(fig_sz_x, fig_sz_y), dpi= fig_dpi, facecolor='w', edgecolor='k')

plt.plot(w_iir / np.pi * nyq_frec, angles_iir, label='IIR-Butter')
plt.plot(w_fir / np.pi * nyq_frec, angles_fir, label='FIR-Win')

plt.title('Filtros diseñado por métodos directos')
plt.xlabel('Frequencia [Hz]')
plt.ylabel('Fase [rad]')
plt.axis('tight')

plt.grid()

axes_hdl = plt.gca()
axes_hdl.legend()

plt.show()

plt.figure(figsize=(fig_sz_x, fig_sz_y), dpi= fig_dpi, facecolor='w', edgecolor='k')

plt.subplot(311)
plt.title('Digital filter group delay IIR')
plt.plot(w_iir / (np.pi * nyq_frec), gd_iir, label='IIR-Group Delay' )
plt.ylabel('Group delay [samples]')
plt.xlabel('Frequency [rad/sample]')
plt.subplot(313)
plt.title('Digital filter group delay FIR')
plt.plot(w_fir / (np.pi * nyq_frec), gd_fir, label='FIR-Group Delay' )
plt.ylabel('Group delay [samples]')
plt.xlabel('Frequency [rad/sample]')
plt.show()

C:\Users\ranuc\AppData\Local\Temp/ipykernel_7624/276289291.py:55: RuntimeWarning: divide by zero encountered in log10
  plt.plot(w_iir/ np.pi * nyq_frec, 20 * np.log10(abs(h_iir)), label='IIR-Butter' )


In [26]:
import matplotlib.pyplot as plt
import scipy.signal as sig
from splane import pzmap
from matplotlib.ticker import FuncFormatter, MultipleLocator

num = [0.009456, -0.07234, 0.2451, -0.4806, 0.5968, -0.4806, 0.2451, -0.07234, 0.009456]
den = [1, -7.705, 25.98, -50.09, 60.37, -46.59, 22.48, -6.2, 0.7485]
fs = 1000
nyq_frec = fs / 2

transfer_f = sig.TransferFunction(num, den, dt=1/fs)

w, h = sig.freqz(num, den)

plt.plot(w / (np.pi * nyq_frec), 20 * np.log10(abs(h)))
# fig, ax1 = plt.subplots()
# ax1.set_title('Respuesta en frecuencia del filtro digital')
# ax1.plot(w, 20 * np.log10(abs(h)), 'b')
# ax1.set_ylabel('Modulo [dB]', color='b')
# ax1.set_xlabel('Frecuencia [rad/muestra]')
# ax1.xaxis.set_major_formatter(FuncFormatter(lambda val,pos: '{:.0g}$\pi$'.format(val/np.pi) if val !=0 else '0'))

# ax2 = ax1.twinx()
# angles = np.unwrap(np.angle(h))
# ax2.plot(w, angles, 'g')
# ax2.set_ylabel('Fase (radianes)', color='g')
# ax2.grid()
# ax2.axis('tight')
# plt.show()

# filter_names=("  ")

# analog_fig_id, analog_axes_hdl = pzmap(transfer_f, filter_names, 3)